<a href="https://colab.research.google.com/github/teixeirabrenno/agente-IA/blob/main/agenteIA-fornecedores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.colab import userdata
from IPython.display import HTML, Markdown
from google import genai
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
client = genai.Client()
MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
##########################################
# --- Agente 1: Buscador de Fornecedores --- #
##########################################

def agente_buscador(topico, data_de_hoje, regiao):
  buscador = Agent(
      name='agente_buscador',
      model='gemini-2.0-flash',
      instruction = '''Você é um agente que busca informações de fornecedores de insumos alimentícios para uma Unidade de Alimentação e Nutrição (UAN).
      Seu dever é buscar os fornecedores, baseados na sua avaliação e custo-benefício. Dê prioridade ao custo-benefício, mas caso a avaliação da
      empresa fornecedora for abaixo de quatro estrelas, dê preferência a outra empresa, mesmo que o valor seja um pouco mais caro. Dê preferência
      a fornecedores próximos ao local da UAN.
      As informações e preços dos insumos devem estar atualizadas, com no máximo duas semanas antes da data de hoje.
      Mostre os 5 melhores fornecedores da região baseados nos critérios acima. Cite o endereço, telefone, WhatsApp e email de contato deles.''',
      description = 'Agente que busca informações sobre fornecedores de insumos alimentares no Google',
      tools = [google_search],
  )

  entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}\nLocalização: {local_de_busca}\nLocal da UAN: {local_UAN}"
  # Executa o agente
  informacoes_fornecedores = call_agent(buscador, entrada_do_agente_buscador)
  return informacoes_fornecedores


In [ ]:
#Informações passadas ao agente

data_de_hoje = date.today().strftime("%d/%m/%Y")
local_de_busca = 'Rio de Janeiro - Rio de Janeiro'
local_UAN = 'Barra da Tijuca, Rio de Janeiro - RJ, 22640-100'
topico = input("Por favor, digite o insumo o qual você deseja pesquisar: ")


if not topico:
  print('Você não escolheu nenhum insumo ainda.')
else:
  print(f'Pesquisando sobre fornecedores de {topico.lower()}!')

  informacoes_fornecedores = agente_buscador(topico, data_de_hoje, local_de_busca)


  print('------- Resultado do Agente Bsucador --------')
  display(to_markdown(informacoes_fornecedores))
  print('--------------------------------------------------')



